# Unprofiled Expectation Maximisation

This notebook is used to show how the **Unprofiled Expectation Maximization** can be used in practice.

This presents practically the work of **TO COMPLETE IF ACCEPTED**
                            
This is a joint work with:
 * Julien Béguinot (Télécom Paris, Institut Polytechnique de Paris)
 * Wei Cheng (Secure-IC S.A.S, Télécom Paris, Institut Polytechnique de Paris)
 * Sylvain Guilley (Secure-IC S.A.S, Télécom Paris, Institut Polytechnique de Paris)
 * Olivier Rioul (Télécom Paris, Institut Polytechnique de Paris)

#### Importation of Required Libraries

In [12]:
import numpy as np
from numpy.linalg import norm

import scipy
from scipy.optimize import minimize
import scipy.io


import matplotlib.pyplot as plt
from matplotlib import cm


from time import time

#### Sbox

Computation of the Substitution Box and Hamming Weight
 * DES Sbox
 * PRESENT Sbox
 * SubByte from AES

In [22]:
binary_repr_vec = np.vectorize(np.binary_repr)
hw = np.char.count

def Sbox_(b):
    
    """
    # 1st line of 1st DES Sbox (can be critized as an NSA cipher, which non-formalized security proof) 
    # https://en.wikipedia.org/wiki/S-box
    S = np.array([2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],dtype=np.uint8) 
    """
    
    # PRESENT
    S = np.array([0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD, 0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2],dtype=np.uint8)

    # SubBytes look-up-table (AES)
    """"
    S = np.array([
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16 ],dtype=np.uint8)
    """

    return S[b]

Sbox = np.vectorize(Sbox_)

In [26]:
M = np.arange(2**4,dtype = np.uint8)
N = 4
Nparams = int(N*(N+1)/2) + 1

#If you target the DPAv4.2 contests. 
#mask = np.array([3,12,53,58,80,95,102,105,150,153,160,175,197,202,243,252],dtype= np.uint8)
#def Mask_(b):
#    return mask[b]
#Mask = np.vectorize(Mask_)

### Synthetic Trace Generation

#### Set Parameters

In [42]:
Q = 200          # Number of Traces
sigma=1          # Level of Noise
sigma_a = .8     # Level of epistemic Noise

#### Draw secret keys, masks, plaintext randomly

In [43]:
secret_k = np.random.randint(0,2**N)                  #The secret key
T = np.random.randint(0,2**N,Q ,dtype=np.uint8)       #The plaintext
Mask = np.random.randint(0,2**N,Q ,dtype=np.uint8)    #The Mask

print("The selected secret key is : ",secret_k)

The selected secret key is :  9


#### Draw random leakage parameters for the quadratic leakage model

In [44]:
secretC0 =  np.ones(Nparams) + sigma_a * np.random.randn(Nparams)
secretC0 *= np.sqrt(Nparams) / norm(secretC0)

secretC1 = np.ones(Nparams) + sigma_a * np.random.randn(Nparams)
secretC1 *= np.sqrt(Nparams) / norm(secretC1)

#### Draw random Additive White Gaussian Noise

In [45]:
Noise0 = sigma * np.random.randn(Q)
Noise1 = sigma * np.random.randn(Q)

#### Generate Sensitive Variables

In [46]:
X1 = np.unpackbits(Sbox(np.uint8(secret_k)^T)^Mask).reshape((Q,8))[:,8-N:8]
X0 = np.unpackbits(Mask).reshape((Q,8))[:,8-N:8]

#### Compute the  noisy leakages with the model

In [47]:
Y0 = np.einsum("qj,ji,qi->q",X0,vec_to_sym(secretC0[0:Nparams-1]),X0) + secretC0[Nparams-1]
Y0 += Noise0

Y1 = np.einsum("qj,ji,qi->q",X1,vec_to_sym(secretC1[0:Nparams-1]),X1) + secretC1[Nparams-1]
Y1 += Noise1

Y = np.array([Y0,Y1])

## Distinguishers 

### 2O-CPA : Second Order Corelation Power Analysis

* This distinguisher ranks the key hypothesis by deacreasing Pearsons correlation coefficient in between the model and the traces.
* The combination function is the classical centered product of the traces.

A reference : *"Emmanuel Prouff, Matthieu Rivain, and Régis Bevan. Statistical Analysis of Second
Order Differential Power Analysis. IEEE Trans. Computers"*

In [48]:
def HO_CPA(T,Y):

    Q = len(T)
    Text,Mask = np.meshgrid(T, M, copy=False, sparse=True)

    ybar = np.reshape(np.mean(Y,axis=1),(2,1))
    Yp = np.prod(Y - ybar,axis=0) #Belong to R^Q

    Pearsons = np.zeros(2 ** N)

    mu_y = np.mean(Yp) #Precompute the mean
    for k in range(2**N):

        X = (hw(binary_repr_vec(Sbox(k^Text)^Mask),'1')-N/2)*(hw(binary_repr_vec(Mask),'1')-N/2)
        mu_x = np.mean(X)
        std_x = np.std(X)

        if std_x == 0:
            print("Warn on cpa with 0 variance")
            return np.arange(16)

        mu_y = np.mean(Yp)
        cov = np.mean((Yp-mu_y )* (X-mu_x))
        Pearsons[k] = abs(cov) / std_x

    k_guess = np.argsort(-Pearsons)
    return k_guess

### Test the distinguisher

In [52]:
k_guess = HO_CPA(T,np.copy(Y))

print("The secret key is ranked ", np.argmax(k_guess == secret_k), " by the distinguisher.")

The secret key is ranked  0  by the distinguisher.


### U-EM-HAM

* This distinguisher regress the parameters of of a stochastic attack on the fly. Then it ranks the different key hypothesis by decreasing goodness of fit.
* The leakage model considered is the Hamming Weight of the sensitive variables.

In [15]:
"""
Subfonction used in the maximization step of the U-EM with HW leakage model.

Input:
    beta : posterior probabilities for each masks and keys
    X    : sensitive varioables of the model
    Y    : leakage
    Q    : number of traces

Output;
    na,b : parameters that maximizes the goodness of fit
"""
def minimize_u(beta,X,Y,Q):
    x_tilde = np.sum(beta * X, axis = 0).T #Should belong to R^Q
    x_bar = np.mean(x_tilde,axis = 0) #Should belong to R
    var_x = (np.sum(beta * (X-x_bar) ** 2) / Q)
    cov = np.sum(beta * (X-x_bar) *Y) / Q
    na = cov / var_x
    b = - na * x_bar
    return na,b

"""
This implements the U-EM-HAM algorithm described in the article.

Input:
    T         : the plaintext used for the considered Traces
    Y         : the two shares of the leakage
    sigma     : the noise level
    tolerance : a threshold to decide when to stopp the while loop

Output:
    k_guess   : the key hypothesis ranked by deacreasing goodness of fit with the model
"""
def EM_bivarie_2(T,Y,sigma=np.reshape(np.array([1,1]),(2,1)),tolerance=10**-10):

    Q = len(T)

    #Normalisation and centering
    Y/=(2 ** .5 * sigma)
    y_bar = np.reshape(np.mean(Y,axis=1),(2,1)) #Precompute the average of the traces
    Y-=y_bar
    Y = np.reshape(Y,(2,1,Q))

    LL = np.zeros(2**N) #To store the log-likelihood of each key hypothesis

    Text,Mask = np.meshgrid(T, M, copy=False, sparse=True)

    X0 = np.reshape(np.char.count(binary_repr_vec(Mask), '1'),(len(Mask),1))

    for k in range(2**N):
        a1,a2 = 1,1
        b1,b2 = 0,0

        X1 = np.char.count(binary_repr_vec(Sbox(k^Text)^Mask), '1')

        stop = False
        while not stop:

            #The E Step
            Norm =(Y[0]-a1*X0-b1)**2 + (Y[1]-a2*X1-b2)**2
            C = np.min(Norm,axis=0) #For numerical stabiblity of the exponnetia
            beta = np.exp(C-Norm)
            S  = np.sum(beta,axis=0) #Normalisation coefficient
            beta = beta / S #Normalised p.m.f

            #The M Step
            na1,b1 = minimize_u(beta,X0,Y[0],Q)
            na2,b2 = minimize_u(beta,X1,Y[1],Q)

            #Does the new values of a and b changed up to a certain tolerance level ?
            stop = abs(na1-a1) + abs(na2-a2) < tolerance

            a1,a2 = na1,na2

        LL[k] = np.sum(np.log(S)) - np.sum(C) #Store the log-likelihood of the key
    k_guess = np.argsort(-LL)
    return k_guess

In [54]:
k_guess = EM_bivarie_2(T,np.copy(Y),sigma=np.reshape(np.array([sigma,sigma]),(2,1)),tolerance=10**-10)

print("The secret key is ranked ", np.argmax(k_guess == secret_k), " by the distinguisher.")

The secret key is ranked  0  by the distinguisher.


### U-EM-LIN

* This distinguisher regress the parameters of of a stochastic attack on the fly. Then it ranks the different key hypothesis by decreasing goodness of fit.
* The leakage model considered is a linear combination of the bits of the sensitive variables.

In [55]:
def minimize_ridge(beta,X,Y,Mask,Q,N,lamb =10**2):
        
    XX = np.sum(X,axis=2)
    x_tilde = np.sum(beta * XX, axis = 0).T #Should belong to R^Q
    x_bar = np.mean(x_tilde,axis = 0) #Should belong to R

    var_x = np.sum(beta * (XX-x_bar) ** 2)
    cov = np.sum(beta * (XX-x_bar) *Y)

    na = cov / var_x * np.ones(N)

    x_tilde = np.sum(np.transpose(X,axes=(2,0,1)) * beta, axis = 1).T #Should belong to R^(QxN)
    x_bar = np.mean(x_tilde,axis = 0) #Should belong to R^N

    X_cent = np.reshape(X - x_bar,(len(Mask),Q,N,1))

    AutoCorr = np.einsum('mq,mqtp->tp' , beta,np.einsum('ijkl,ijhl->ijkh', X_cent,X_cent))  + Q *  lamb * np.eye(N)
    X_cent = np.reshape(X_cent,(len(Mask),Q,N))

    RelCorr = np.einsum('qm,q->m' ,  x_tilde ,np.reshape(Y - np.dot(x_tilde-x_bar,na),(Q)))  #Since Y is centered no need to sub x_bar to x_tilde

    na += np.linalg.solve(AutoCorr,RelCorr)
    #na += np.linalg.lstsq(AutoCorr,RelCorr,rcond=None)[0]
    nb = - np.dot(x_bar,na)

    return na,nb


def EM_BIV_LIN_RIDGE(T,Y,N=4,tolerance=10**-8,sigma=np.reshape(np.array([1,1]),(2,1))):
        
        t1 = time()
        lamb = 1/(2 * sigma_a ** 2)
        Q = len(T)

        Y/=(2 ** .5 * sigma)
        y_bar = np.reshape(np.mean(Y,axis=1),(2,1)) #Precompute the average of the traces
        Y-=y_bar
        sigma_y=np.std(Y,axis=1)

        LL = np.zeros(2**N) #To store the log-likelihood of each key hypothesis

        #M = np.array([3,12,53,58,80,95,102,105,150,153,160,175,197,202,243,252],dtype = np.uint8)
        Text,Mask = np.uint8(np.meshgrid(T, M))

        X2 = np.reshape(np.unpackbits(Mask),(2**N,Q,8))[:,:,8-N:8]

        for k in range(2**N):
            
            #a_1,b_1 = np.ones(N) * np.abs( sigma_y[0] ** 2 - sigma[0] **2) ** .5 * np.sqrt(4/N),0
            #a_2,b_2 = np.ones(N) * np.abs( sigma_y[1] ** 2 - sigma[1] **2) ** .5 * np.sqrt(4/N),0
            a_1,b_1 = np.ones(N),0
            a_2,b_2 = np.ones(N),0

            X1 = np.reshape(np.unpackbits(Sbox(k^Text)^Mask),(2**N,Q,8))[:,:,8-N:8]

            stop = False                                                                  
            while not stop:
                #The E Step
                Norm = (Y[0,:]-np.dot(X1,a_1)-b_1)**2+(Y[1,:]-np.dot(X2,a_2)-b_2)**2
                C = np.min(Norm,axis=0) #For numerical stabiblity of the exponnetial
                beta = np.exp(C-Norm)
                S  = np.sum(beta,axis=0) #Normalisation coefficient
                beta = beta / S #Normalised p.m.f

                #The M Step
                na1,b_1 = minimize_ridge(beta,X1,Y[0,:],Mask,Q,N,lamb=lamb)
                na2,b_2 = minimize_ridge(beta,X2,Y[1,:],Mask,Q,N,lamb=lamb)

                #Have a and b changed up to a certain tolerance level ?
                diff = np.linalg.norm(na1-a_1)**2  + np.linalg.norm(na2-a_2)**2
                stop = (diff < tolerance)
                
                a_1,a_2 = na1,na2
                #print(a_1)
                
            LL[k] = np.sum(np.log(S)) - np.sum(C) #Store the log-likelihood of the key
            
        k_guess = np.argsort(-LL)
        
        t2 = time()
        
        return k_guess

### Test the distinguisher

In [56]:
k_guess = EM_bivarie_2(T,np.copy(Y),sigma=np.reshape(np.array([sigma,sigma]),(2,1)),tolerance=10**-10)

print("The secret key is ranked ", np.argmax(k_guess == secret_k), " by the distinguisher.")

The secret key is ranked  0  by the distinguisher.


### U-EM-QUAD

* This distinguisher regress the parameters of of a stochastic attack on the fly. Then it ranks the different key hypothesis by decreasing goodness of fit.
* The leakage model considered is a quadratic function of the bits of the sensitive variables.

In [57]:
"""
Subfonction used for the U-EM-QUAD
Input:
    Cvec : a list of Nparams - 1 parameters
Output:
    sym  : an upper trangular matrix where the parameters are layed out
"""
def vec_to_sym(Cvec):
    pos = 0
    sym = np.zeros((N,N))
    for i in range(N):
        sym[i,i:] =  Cvec[pos:pos+N-i]
        pos+=N-i
    return sym

"""
Another subfonction that does the inverse of the previous one.
(with some extra dimension at the begining)
"""
def matrix_to_vec(to_be_vec,m,q):
    pos = 0
    vec = np.zeros((m,q,Nparams-1))
    for i in range(N):
        vec[:,:,pos:pos+N-i] = to_be_vec[:,:,i,i:N]
        pos+=N-i
    return vec

"""
Subfonction that evaluates a quadratic form with design matrix C at the vector X.
"""
def evaluate(C,X):
    return np.einsum("mqj,ji,mqi->qm",X,vec_to_sym(C[0:Nparams-1]),X) + C[Nparams-1]

"""
This implements the U-EM-QUAD as described in the article.

Input :
    T          :  the plaintext used with the considered traces
    Y          :  the leakages
    sigma      :  the noise level
    regu       :  the constant used for regression ridge
    tolerance  :  threshold to decide when to stop the while loop

Output:
    k_guess    : the key hypothesis ranked by deacreasing goodness of fit with the model.
"""
def EM_quadratic(T,Y,sigma=np.reshape(np.array([1,1]),(2,1)),regu = 1,tolerance=5*10**-3):

    Q = len(T)

    #Normalisation and centering
    Y/=(2 ** .5 * sigma)

    ones = np.ones(Nparams) / (2 ** .5 * sigma[0])

    sigma_y = np.std(Y,axis=1)
    y_bar = np.reshape(np.mean(Y,axis=1),(2,1)) #Precompute the average of the traces
    Y-=y_bar
    Y = Y.reshape((2,Q))

    LL = np.zeros(2**N) #To store the log-likelihood of each key hypothesis

    #Text,Mask = np.meshgrid(T, M, copy=False, sparse=True)
    Text,Mask = np.meshgrid(T,M, copy = False)

    X0 = np.unpackbits(Mask).reshape((2**N,Q,8))[:,:,8-N:8]
    X0vec = np.ones((2**N,Q,Nparams))
    X0vec[:,:,0:Nparams-1] = matrix_to_vec(np.einsum("abn,abm->abnm",X0,X0),2**N,Q)
    X0vec = X0vec.transpose((1,0,2))

    for k in range(2**N):

        Cvec0 = np.copy(ones)
        Cvec1 = np.copy(ones)

        X1 = np.unpackbits(Sbox(k^Text)^Mask).reshape((2**N,Q,8))[:,:,8-N:8]
        X1vec = np.ones((2**N,Q,Nparams))
        X1vec[:,:,0:Nparams-1] = matrix_to_vec(np.einsum("abn,abm->abnm",X1,X1),2**N,Q)
        X1vec = X1vec.transpose((1,0,2))

        last_ll = 10**4 #dummy init

        stop = False
        maxiter = 15
        niter = 0
        while (not stop) and (niter < maxiter):

            niter += 1

            # Precompute the Quadratic Form
            f0 = evaluate(Cvec0,X0).reshape((Q,2**N))
            f1 = evaluate(Cvec1,X1).reshape((Q,2**N))

            #The E Step
            Norm =  (Y[0].reshape((Q,1)) - f0)**2
            Norm += (Y[1].reshape((Q,1)) - f1)**2 #Shape (Q,2**N)

            #For numerical stabiblity of the exponential
            Cc = np.min(Norm,axis=1).reshape((Q,1))#Most likely mask for each trace

            #Compute pdf with Bayes
            beta = np.exp(Cc-Norm) #Gaussian Kernel with shape (Q,2**N)
            S  = np.sum(beta,axis=1) #Normalisation coefficient in Bayes
            beta = beta / S.reshape((Q,1))#Normalised p.m.f

            #The M Step
            def objective0(Cvec):
                return np.sum(beta * (Y[0].reshape((Q,1)) - evaluate(Cvec,X0).reshape((Q,2**N))) ** 2) + regu * Q * norm(Cvec-ones)

            def jac0(Cvec):
                return - 2 * np.sum(beta.reshape((Q,2**N,1)) * (Y[0].reshape((Q,1,1)) - evaluate(Cvec,X0).reshape((Q,2**N,1))) * X0vec,axis=(0,1)) + regu * 2 * Q * (Cvec-ones)

            def objective1(Cvec):
                return np.sum(beta * (Y[1].reshape((Q,1)) - evaluate(Cvec,X1).reshape((Q,2**N))) ** 2) + regu *  Q * norm(Cvec-ones)
            def jac1(Cvec):
                return - 2 * np.sum(beta.reshape((Q,2**N,1)) * (Y[1].reshape((Q,1,1)) - evaluate(Cvec,X1).reshape((Q,2**N,1))) * X1vec,axis=(0,1)) + regu * 2 * Q * (Cvec-ones)

            #Update parameters
            Cvec0 = minimize(objective0,Cvec0,method="BFGS",jac=jac0,options={'gtol': 1e-04, 'maxiter': 10}).x
            Cvec1 = minimize(objective1,Cvec1,method="BFGS",jac=jac1,options={'gtol': 1e-04, 'maxiter': 10}).x

            #Does the new values of a and b changed up to a certain tolerance level ?
            ll = ((np.sum(np.log(S)) - np.sum(Cc))/Q)
            stop = np.abs(ll-last_ll) < tolerance

            #Store obtained goodness of fit
            last_ll = ll

        LL[k] = np.sum(np.log(S)) - np.sum(Cc) #Store the log-likelihood of the key

    k_guess = np.argsort(-LL)

    return k_guess


### Test the distinguisher

In [58]:
k_guess = EM_bivarie_2(T,np.copy(Y),sigma=np.reshape(np.array([sigma,sigma]),(2,1)),tolerance=10**-10)

print("The secret key is ranked ", np.argmax(k_guess == secret_k), " by the distinguisher.")

The secret key is ranked  0  by the distinguisher.


### Template HAM

In [18]:
"""
This implements the  template attack with Hamming model.

Input:
    T         : the plaintext used for the considered Traces
    Y         : the two shares of the leakage
    sigma     : the noise level
    tolerance : a threshold to decide when to stopp the while loop
    N         : number of bits targeted
    a_1,...   : parameters of the template to use

Output:
    k_guess   : the key hypothesis ranked by deacreasing goodness of fit with the model
"""
def template_ham(T,Y,sigma=np.reshape(np.array([1,1]),(2,1)),a_1=1,b_1=0,a_2=1,b_2=0,N=4):

    t1 = time()

    a1=a_1/(2 ** .5 * sigma[0])
    a2=a_2/(2 ** .5 * sigma[1])
    b1=b_1/(2 ** .5 * sigma[0])
    b2=b_2/(2 ** .5 * sigma[1])

    Q = len(T)

    Y/=(2 ** .5 * sigma)
    Y = np.reshape(Y,(2,1,Q))


    LL = np.zeros(2**N) #To store the log-likelihood of each key hypothesis

    Text,Mask = np.meshgrid(T, M, copy=False, sparse=True)

    X1 = np.reshape(np.char.count(binary_repr_vec(Mask), '1'),(len(Mask),1))

    for k in range(2**N):

        X0 = np.char.count(binary_repr_vec(Sbox(k^Text)^Mask), '1')

        Norm =(Y[0]-a1*X0-b1)**2 + (Y[1]-a2*X1-b2)**2

        C = np.min(Norm,axis=0) #For numerical stabiblity of the exponnetial
        beta = np.exp(C-Norm)
        S  = np.sum(beta,axis=0) #Normalisation coefficient
        beta/=S #Normalised p.m.f

        LL[k] = np.sum(np.log(S)) - np.sum(C) #Store the log-likelihood of the key

    k_guess = np.argsort(-LL)

    t2 = time()

    return k_guess

### Template LIN

In [19]:
"""
This implements the  template attck with linear leakage model.

Input:
    T         : the plaintext used for the considered Traces
    Y         : the two shares of the leakage
    sigma     : the noise level
    tolerance : a threshold to decide when to stopp the while loop
    N         : number of bits targeted
    a_1,...   : parameters of the template to use

Output:
    k_guess   : the key hypothesis ranked by deacreasing goodness of fit with the model
"""
def template2(T,Y,sigma=np.reshape(np.array([1,1]),(2,1)),a_1=np.ones(4),b_1=0,a_2=np.ones(4),b_2=0,N=4):

    a1=a_1/(2 ** .5 * sigma[0])
    a2=a_2/(2 ** .5 * sigma[1])
    b1=b_1/(2 ** .5 * sigma[0])
    b2=b_2/(2 ** .5 * sigma[1])

    Q = len(T)
    Y/=(2 ** .5 * sigma)

    y_bar = np.mean(Y,axis=1) #Precompute the average of the traces
    LL = np.zeros(2**N) #To store the log-likelihood of each key hypothesis

    Text,Mask = np.uint8(np.meshgrid(T, M))

    X2 = np.reshape(np.unpackbits(Mask),(2**N,Q,8))[:,:,8-N:8]

    for k in range(2**N):

        X1 = np.reshape(np.unpackbits(Sbox(k^Text)^Mask),(2**N,Q,8))[:,:,8-N:8]
        Norm = (Y[0,:]-np.dot(X1,a1)-b1)**2+(Y[1,:]-np.dot(X2,a2)-b2)**2
        C = np.min(Norm,axis=0) #For numerical stabiblity of the exponnetial
        beta = np.exp(C-Norm)
        S  = np.sum(beta,axis=0) #Normalisation coefficient

        LL[k] = np.sum(np.log(S)) - np.sum(C) #Store the log-likelihood of the key


    k_guess = np.argsort(-LL)
    return k_guess

### Template QUAD

In [20]:

"""
This implements a template attack with quadratic leakage model.
Input:
    T      : the plaintext used for the considered traces.
    Y      : the traces
    sigma  : the noise level
    Cvec0  : model parameters (Quadratic form of the first share)
    Cvec1  : model parameters (Quadratic form of the second share)
"""
def template_quadratic(T,Y,sigma,Cvec0,Cvec1):

    Q = len(T)

    Y/=(2 ** .5 * sigma)
    Cvec0/=(2 ** .5 * sigma[0])
    Cvec1/=(2 ** .5 * sigma[1])

    LL = np.zeros(2**N) #To store the log-likelihood of each key hypothesis
    Text,Mask = np.meshgrid(T,M, copy = False)
    X0 = np.unpackbits(Mask).reshape((2**N,Q,8))[:,:,8-N:8]

    for k in range(2**N):
        X1 = np.unpackbits(Sbox(k^Text)^Mask).reshape((2**N,Q,8))[:,:,8-N:8]

        # Precompute the Quadratic Form
        f0 = evaluate(Cvec0,X0).reshape((Q,2**N))
        f1 = evaluate(Cvec1,X1).reshape((Q,2**N))

        Norm =  (Y[0].reshape((Q,1)) - f0)**2
        Norm += (Y[1].reshape((Q,1)) - f1)**2 #Shape (Q,2**N)

        Cc = np.min(Norm,axis=1).reshape((Q,1))#Most likely mask for each trace
        beta = np.exp(Cc-Norm) #Gaussian Kernel with shape (Q,2**N)
        S  = np.sum(beta,axis=1) #Normalisation coefficient in Bayes

        LL[k] = np.sum(np.log(S)) - np.sum(Cc) #Store the log-likelihood of the key
    k_guess = np.argsort(-LL)

    return k_guess
